In [1]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint

In [2]:
# make page
paper_size = '14x11 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
lines = []

node_sets = []
n_lines = 500
n_nodes_per_line = 40
y_start = 0
y_end = 14
x_start = 0
x_end = 10
node_x_centers = np.linspace(x_start, x_end, n_lines)
std_scale = 0.09
n_eval_points = 80

### initialize vals
node_ys = np.linspace(y_start, y_end, n_nodes_per_line)
centered_node_xs = np.zeros(node_ys.shape)
bez_eval_end_center = 1
bez_eval_end_noise = 0
bez_eval_end_limit = 1.
bez_eval_end_std_scale = 0.01

for i, node_x_center in enumerate(node_x_centers):
    new_x_noise = np.random.randn(n_nodes_per_line) * std_scale
    centered_node_xs = centered_node_xs + new_x_noise
    node_xs = node_x_center + centered_node_xs
    node_xs[:3] = node_x_center
    node_xs[-3:] = node_x_center
    nodes = np.asfortranarray([
        node_xs,
        node_ys,
        ])
    curve = bezier.Curve(nodes, degree=(nodes.shape[1]-1))
    eval_start = np.random.uniform(0, 0.03)
    eval_end = np.random.uniform(0.97, 1.)
    eval_points = np.linspace(eval_start, eval_end, n_eval_points)
    x, y = curve.evaluate_multi(eval_points)
    if i % 2:
        x = np.flipud(x)
        y = np.flipud(y)
    lines.append(np.stack([x, y]).T)
    node_sets.append(np.stack([node_xs, node_ys]).T)

In [ ]:
ls = [LineString(l) for l in lines]

mls = gp.make_like(gp.merge_LineStrings(ls), drawbox)

mask = drawbox

in_mask = mls.intersection(mask)

in_mask = sa.rotate(in_mask, -90)

split_point = 500

layer1 = in_mask[:split_point]
layer2 = in_mask[split_point:]

layers = []
layers.append(LineString(np.concatenate([np.array(l) for l in layer1])))
layers.append(LineString(np.concatenate([np.array(l) for l in layer2])))

# layers = [in_mask]

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesort')

sk.display(color_mode='none')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/246_500_lines.svg')

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=55
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [2]:
def oscillator(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v) + (a * v) + (b * u)
    dudt = np.cos(u) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

def oscillator2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v) + np.sin(u) + (a * v) + (b * u)
    dudt = np.cos(u) + np.cos(u) ** 2 + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [3]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v)  + np.cos(u * v) + (a * v) + (b * u)
    dudt = np.cos(u) + np.sin(u * v) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 500
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(.5, 4.5, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius
    y = np.sin(theta) * radius
    pts.append(Point(x, y))

In [ ]:
lfs = []


t_max = 3.7
t = np.linspace(0, t_max, 1801)
a = -0.4
b = 0.3
c = 0.75
d = -0.2



for pt in tqdm(pts):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lines)

sk.penWidth('0.3')
sk.vpype('linesimplify linesort ')

sk.display(color_mode='none')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/247_500_lines.svg')

# Try 2

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=55
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 500
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(.75, 3.45, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 3.3
    y = np.sin(theta) * radius + 0.5
    pts.append(Point(x, y))

In [ ]:
def ode2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(u * v + (a * v) + (b * u))
    dudt = np.cos(u) + np.sin(u * v) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
lfs = []


t_max = 2.7
t = np.linspace(0, t_max, 1801)
a = -0.2
b = -0.2
c = 0.04
d = -0.25



for pt in tqdm(pts):
    sol = odeint(ode2, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

In [ ]:
layers = []
layers.append(lines[:250])
layers.append(lines[250:])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/249_500_lines.svg')

# Try 3

In [4]:
# make page
paper_size = '17x23.5 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [5]:
center = drawbox.centroid

In [6]:
n_lines = 3500
thetas = np.linspace(0, np.pi*14, n_lines)
radii = np.linspace(.5, 5.45, n_lines)

In [7]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 3.3
    y = np.sin(theta) * radius + 0.5
    pts.append(Point(x, y))

In [8]:
def ode2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(u * v + (a * v) + (b * u))
    dudt = np.cos(u) + np.sin(u * v) + np.cos(c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [9]:
lfs = []


t_max = 2.7
t = np.linspace(0, t_max, 701)
a = -0.2
b = -0.25
c = 0.04
d = -0.25



for pt in tqdm(pts):
    sol = odeint(ode2, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

100%|██████████| 3500/3500 [00:05<00:00, 621.47it/s]


In [25]:
lbs = lines.buffer(0.07, cap_style=2, join_style=2).boundary

In [26]:
lbs = gp.merge_LineStrings([l for l in lbs if l.length > 0.9])

In [27]:
n_layers = 1
layer_inds = np.split(np.arange(len(lbs)), n_layers)

In [28]:
layers = []
for ind_set in layer_inds:
    layer = [lbs[i] for i in ind_set]
    layers.append(gp.merge_LineStrings(layer))
    

In [29]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify')

sk.display(color_mode='layer')

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-27T12:01:54.801614 
 
 
 
 
 
 <polygon points="1200.5767,1322.828 1201.6616,1327.8136 1203.1317,1332.3416 1205.0544,1336.5654 1207.4803,1340.5892 1210.5454,1344.6068 1214.1717,1348.5028 1218.4715,1352.3878 1223.1789,1356.0199 1228.6227,1359.6113 1234.1112,1362.6835 1239.6564,1365.2852 1245.4107,1367.4772 1251.2186,1369.1712 1256.92,1370.3195 1262.3701,1370.9247 1267.9175,1371.0237 1276.6957,1370.0438 1285.2074,1367.6511 1293.6484,1363.7442 1301.8173,1358.4043 1309.6094,1351.8455 1317.9242,1343.3508 1326.5542,1333.1855 1337.9333,1318.5461 1325.667,1335.0911 1317.202,1345.6941 1309.0829,1354.6555 1301.4946,1361.6859 1293.544,1367.5488 1285.7388,1371.8255 1278.3186,1374.6104 1273.3537,1375.8248 1267.8556,1376.6033 1267.905,1377.1301 1276.2021,1375.7349 1284.665,1372.8797 1292.6257,1368.748 1300.7351,1362.9946 1308.1274,1356.3603 1316.1041,1347.7949 1324.7309,1337.1781 1335.9008,1322.2142 1323.9671,1338.9593 1315.5171,1350.0086 1307.7398,1359.0194 1300.1924,1366.4201 1291.8822,1372.9149 1283.6451,1377.6666 1275.7471,1380.7501 1270.3032,1382.1028 1264.3342,1382.9175 1264.379,1383.4447 1273.1655,1382.0122 1281.7008,1379.131 1290.1638,1374.6676 1298.3428,1368.7063 1306.1309,1361.4733 1314.1262,1352.5427 1322.4921,1341.8475 1333.8805,1325.996 1321.6948,1343.7448 1313.497,1354.8553 1305.6955,1364.2205 1298.4758,1371.5739 1294.0378,1375.4219 1289.7592,1378.6456 1285.2806,1381.5204 1281.0449,1383.7863 1276.6705,1385.6857 1272.1843,1387.1928 1266.7046,1388.4652 1260.7342,1389.1885 1260.7713,1389.7164 1265.4055,1389.2148 1270.0343,1388.3089 1278.6421,1385.4589 1283.0029,1383.3907 1287.213,1380.9672 1295.5095,1374.8951 1303.4053,1367.4636 1311.802,1357.8493 1320.4914,1346.3475 1331.9879,1329.7237 1319.9183,1347.9821 1311.7265,1359.5175 1303.925,1369.2408 1296.6966,1376.8768 1292.2492,1380.8743 1287.5591,1384.5126 1283.0488,1387.4681 1278.3506,1390.0053 1273.4913,1392.0802 1268.9685,1393.5401 1263.0083,1394.8179 1257.0558,1395.4156 1257.0825,1395.9441 1262.1618,1395.4772 1266.7998,1394.6302 1271.4025,1393.3654 1275.9235,1391.6842 1280.3252,1389.6027 1284.5802,1387.1492 1289.0706,1384.0652 1293.352,1380.6323 1301.2967,1372.9561 1309.7319,1362.9996 1318.4465,1351.0631 1330.2192,1333.393 1318.0881,1352.4426 1309.8978,1364.4002 1302.0886,1374.4684 1294.8425,1382.3656 1289.9965,1386.8238 1285.2666,1390.5447 1280.2956,1393.8086 1275.5458,1396.3404 1271.0949,1398.2237 1266.0954,1399.8118 1259.6676,1401.0958 1253.2996,1401.5978 1253.3134,1402.1267 1258.3516,1401.7924 1262.9866,1401.0747 1268.076,1399.8001 1272.6392,1398.1928 1277.0971,1396.1683 1281.8403,1393.4882 1286.3878,1390.3788 1290.7251,1386.8961 1299.115,1378.6737 1307.5995,1368.396 1316.6223,1355.6189 1328.5702,1337.0004 1316.4593,1356.7466 1308.293,1369.1248 1300.4999,1379.5434 1292.8963,1388.0825 1288.0255,1392.6627 1282.8612,1396.7736 1277.8422,1400.0665 1272.6089,1402.8082 1267.6647,1404.7952 1262.6343,1406.2601 1256.2121,1407.381 1249.4659,1407.734 1249.4646,1408.2632 1254.8924,1408.0451 1259.9725,1407.3606 1265.0732,1406.1679 1269.6664,1404.6262 1274.6122,1402.4318 1279.3983,1399.7416 1283.9911,1396.5998 1288.3727,1393.062 1296.8421,1384.6566 1305.6985,1373.6625 1314.7292,1360.4449 1327.0352,1340.5457 1314.7445,1361.3476 1306.5962,1374.1451 1298.8098,1384.9 1291.1995,1393.6995 1285.93,1398.7551 1280.7273,1402.9361 1275.2406,1406.5204 1269.9592,1409.2268 1264.5266,1411.3102 1259.0204,1412.7576 1252.6281,1413.6742 1245.5552,1413.8236 1245.5366,1414.3524 1250.8556,1414.3231 1256.3476,1413.7716 1261.9115,1412.6247 1266.9889,1411.0152 1271.9727,1408.8674 1276.8071,1406.2019 1281.4528,1403.0597 1286.2795,1399.1523 1290.4745,1395.1979 1294.8054,1390.5546 1303.7013,1379.2484 1313.0244,1365.1559 1325.6071,1344.0321 1313.1876,1365.8633 1304.7826,1379.5331 1296.9913,1390.6075 1289.3594,1399.6374 1284.0671,1404.807 1278.8382,1409.0673 1273.3236,1412.7038 1267.5693,1415.634 1262.1125,1417.6682 1256.144,1419.1398 1248.8986,

In [30]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/251_3500_lines.svg')